# 03. Identify Candidate Intersections OSM

Build a CSV file with a list of candidate intersections to sample from Google Street View based purely on an OpenStreetMap XML extract.  We look for "ways" that have the tag "cycleway" and then find any nodes on those ways that are shared with another way.

You can download OpenStreetMap XML data for Australia from:

http://download.geofabrik.de

And use the "osmium" tool to reduce it to a smaller bounding box.

Or you can use the online service:

https://extract.bbbike.org

To select the required extract.  Please consider donating money to help them run that service, if you use it.

The data for Victoria is 2.82GB uncompressed, so it is not included in the GitHub repository.  Please use one of the above service to obtain it.

If downloading from geofabrik, the .osm.bz2 is a compressed version of the XML file you want, but you'll only be able to download Australia as a whole.  I recommend you download the .osm.pbf file -- a smaller file in a non-XML format -- and then use the "osmium" tool to cut it to a bounding box based on a pair of latitude/longitude coordinates.

To install and use "osmium", see:

https://osmcode.org/osmium-tool/

## Configuration

Any configuration that is required to run this notebook can be customized in the next cell

In [1]:
# The name of the output CSV files with a list of intersections to sample,
# based on cycleways in the OSM extract
# Will be saved to the 'data_sources' directory
output_intersections_file = 'osm_intersections.csv'

# Filename of OpenStreetMap XML extract that includes Victoria
# Expected to be found in the 'data_sources' directory
osm_victoria_file = 'planet_victoria.osm'

# Google Street View API key filename
# To download Google Street View images via the API, you must have an API Key as per:
#  https://developers.google.com/maps/documentation/streetview/get-api-key
# linked to your Google account and billing information.  Otherwise they don't know who to charge,
# so you won't be able to download the images.
# Store your API key in a file with the name listed below, in the parent of the current working directory
# from which you launched Jupyter Notebook
# Do not share your API key with anyone else!
gsv_api_key_filename = 'apikey.txt'

## Code

In [2]:
# General imports
import os
import sys

import pandas as pd

from datetime import datetime

from tqdm.notebook import tqdm

import xml.etree.cElementTree as ET
from collections import defaultdict

In [3]:
# Functions to help write log messages that keep track of how long everything took
timestamp_starting = 0

def log_starting(msg):
    global timestamp_starting
    timestamp_starting = datetime.now()
    print(str(timestamp_starting) + ' START - ' + msg, flush=True)

def log_finished(msg):
    global timestamp_starting
    timestamp_finished = datetime.now()
    timestamp_duration = timestamp_finished - timestamp_starting
    print(str(timestamp_finished) + ' END   - ' + msg
        + '(' + str(timestamp_duration.total_seconds()) + ')',
        flush=True
    )

Load raw OpenStreetMap data for the area into memory.  This will be used to find all the intersections for each [local_street] x [town] mentioned in the PBN data.

The extract was downloaded from http://extract.bbbike.org.  Region limited to one small down for initial testing, then a box that encompasses all of Victoria.

The "Victoria" data took around 10 minutes to load into memory, and the resulting Python process used approximately 9GB of memory.

In [ ]:
osm_file_path = os.path.join(os.path.abspath(os.pardir), 'data_sources', osm_victoria_file)

log_starting('Read raw OpenStreetMap data')

# In-Memory caching via dictionary objects
nodes_per_way = defaultdict(list) # List of nodes in each way
ways_per_node = defaultdict(list) # List of ways associated with each node
ways_by_name  = defaultdict(list) # List of ways associated with each street name
ways_by_id    = {}                # List of ways by osm_id
node_lat      = {}                # Latitude of an intersection node by oms_id
node_lon      = {}                # Longitude of an intersection node by oms_id

# Read the OpenStreetMap XML file
context = ET.iterparse(osm_file_path, events=('start', 'end'))
context = iter(context)

way_id  = 0  # Keep track of which "way" object we are reading from XML, 0=none
node_id = 0  # Keep track of which "node" (nd) object we are reading from XML, 0=none

# Iterate through every XML element in the file as it starts or finishes
# This approach allows us to "stream" the XML rather than try to load it all into
# memory at once.  We only cache what is important to us.
for event, elem in context:
    tag   = elem.tag
    value = elem.text
    
    if value:
        value = value.encode('utf-8').strip()
    
    # Process the start of an XML tag
    if event == 'start':
        # Process "way" objects
        if tag == 'way':
            way_id = elem.get('id', 0)
        # Process "node" (nd) objetcts inside (associated with) a "way"
        elif tag == 'nd':
            node_id = elem.get('ref', 0)
            if way_id != 0:
                # Record that this node was inside this way
                nodes_per_way[way_id].append(node_id)
                ways_per_node[node_id].append(way_id)
        # Process "tag" objects that give a street name for each "way"
        elif tag == 'tag':
            if way_id != 0 and elem.get('k', '?') == 'name':
                way_name = elem.get('v', '?')
                ways_by_name[way_name.upper()].append(way_id)
                ways_by_id[way_id] = way_name.upper()
                
    # At the end of an XML tag, if it was a "way" then record that we are no longer
    # in the middle of reading a "way"
    if event == 'end' and tag == 'way':
        way_id = 0

    elem.clear()

log_finished('Read raw OpenStreetMap data')


# Second pass to load the latitude and longitude of nodes IF AND ONLY IF
# they are involved in an intersection

log_starting('Find Lat/Lon for intersections')

context = ET.iterparse(osm_file_path, events=('start', 'end'))
context = iter(context)

way_id  = 0
node_id = 0

for event, elem in context:
    tag   = elem.tag
    value = elem.text
    
    if value:
        value = value.encode('utf-8').strip()
    
    # Process the start of an XML tag
    if event == 'start':
        # Find the latitude/longitude for each "node" by its oms_id
        if tag == 'node':
            node_id = elem.get('id', 0)
            lat     = elem.get('lat', 0)
            lon     = elem.get('lon', 0)
            
            # If and only if this "node" had more than one "way" associated with it,
            # it is part of an "intersection" and therefore we record its latitude
            # and longitude in memory, by its oms_id
            if len(ways_per_node[node_id]) > 1:
                node_lat[node_id] = lat
                node_lon[node_id] = lon

    elem.clear()

log_finished('Find Lat/Lon for intersections')

Define a function to directly call the Nominatim geocode service to turn lat/lon into geocoded information, because we need the "bounding box" part of the data, which is not returned by the Nominatim API.

In [ ]:
cached_nominatim_search = {}

def nominatim_search(street, city):
    # Check local cache first, used cached results if available
    key = street + ' - ' + city
    
    if key in cached_nominatim_search:
        return cached_nominatim_search[key]
    
    # api-endpoint
    URL = 'http://' + nominatim_domain + '/search'
    
    params = {
        'street': street,
        'city': city
    }
        
    # sending get request and saving the response as response object
    r = requests.get(url = URL, params = params)
    
    # Cache results
    cached_nominatim_search[key] = r.json()
    
    # extracting data in json format
    return r.json()

Define a function to determine whether two bounding boxes overlap.  Include a "margin" to account for bounding boxes that almost overlap but are just off by a small margin, the size of an intersection or so.

In [ ]:
def is_overlapping(box1, box2, margin=0.05):
    # Add margin to box1
    box1_margin = []
    box1_margin.append(box1[0] - margin)
    box1_margin.append(box1[1] + margin)
    box1_margin.append(box1[2] - margin)
    box1_margin.append(box1[3] + margin)

    #print('Margin: ' + str(box1_margin))
    
    # Check if latitude or longitude overlaps
    lat_overlap = False
    lon_overlap = False
    
    if box1_margin[0] <= box2[0] <= box1_margin[1]:
        lat_overlap = True
    if box1_margin[0] <= box2[1] <= box1_margin[1]:
        lat_overlap = True
    if box1_margin[2] <= box2[2] <= box1_margin[3]:
        lon_overlap = True
    if box1_margin[2] <= box2[3] <= box1_margin[3]:
        lon_overlap = True
        
    return (lat_overlap and lon_overlap)

Given a [local_street] x [town], find all other roads that intersect with it.

Some roads like "Main Street" might be very common.  To avoid false-positives, we
check the bounding boxes for both the original street and the candidate street that
appears to intersect based on the [local_street] name alone.  If the bounding boxes
overlap, or are close, then we are comfortable that it's not a duplicate street name
from another area.

In [ ]:
def lookup_bounding_box(objectid):
    row = pbn[pbn['objectid']==objectid]
    
    try:
        gs = row['geometry']
        return [gs.bounds['miny'][0], gs.bounds['maxy'][0], gs.bounds['minx'][0], gs.bounds['maxx'][0]]
    except Exception:
        return None
    
def inside_bounding_box(node_id, bounding_box, margin=0.05):
    # Add margin to box1
    box_margin = []
    box_margin.append(bounding_box[0] - margin)
    box_margin.append(bounding_box[1] + margin)
    box_margin.append(bounding_box[2] - margin)
    box_margin.append(bounding_box[3] + margin)
    
    if node_id not in node_lat:
        return False
    if node_id not in node_lon:
        return False
    if not (box_margin[0] <= float(node_lat[node_id]) <= box_margin[1]):
        return False
    if not (box_margin[2] <= float(node_lon[node_id]) <= box_margin[3]):
        return False
    return True

In [ ]:
def find_intersections(street, town, suburb, city, objectid=0, debug=0):
    # Find bounding_box of original street
    
    # First try using information loaded from directly form PBN coordinates
    original_bounding_box = lookup_bounding_box(objectid)
    
    if original_bounding_box is None:
        # Use most specific first town->suburb->city
        # E.g. Aberdeen Road Fyansford might actually be found in Geelong
        original_street = nominatim_search(str(street).upper(), str(town).upper())
    
        if (len(original_street) < 1):
            original_street = nominatim_search(str(street).upper(), str(suburb).upper())
        
            if (len(original_street) < 1):
                original_street = nominatim_search(str(street).upper(), str(city).upper())
            
                if (len(original_street) < 1):
                    return []
    
        original_bounding_box_str = original_street[0]['boundingbox']
        original_bounding_box = [float(i) for i in original_bounding_box_str]
    
    if (debug > 2):
        print('Original:  ' + street.upper() + " = " + str(original_bounding_box))
    
    # Find way_id list for the name
    way_ids = ways_by_name[street.upper()]
    
    intersection_dict = {}
    
    # Find every matching street name (possibly in another suburb!)   
    loop_counter = 0
    
    for way_id in way_ids:
        # Find every node associated with that street name (including other suburbs)            
        for node_id in nodes_per_way[way_id]:
            # Check that this street is in the same area (probably the same suburb)
            if inside_bounding_box(node_id, original_bounding_box):
                # Find every other street the node is associated with                
                for way_id2 in ways_per_node[node_id]:
                    # Ignore any ways that were clipped from the map
                    if way_id2 in ways_by_id:
                        # Find the street name for the other potential intersecting street
                        intersection_name = ways_by_id[way_id2]
                        if intersection_name.upper() != street.upper():                        
                            # Check Nominatim service to see if boundary boxes roughly overlap
                        
                            pot_streets = nominatim_search(intersection_name.upper(), city.upper())
                            for pot_street in pot_streets:
                                pot_street_bounding_box_str = pot_street['boundingbox']
                                pot_street_bounding_box = [float(i) for i in pot_street_bounding_box_str]
                            
                                if (debug > 2):
                                    print('Potential: ' + intersection_name + ' = ' + str(pot_street_bounding_box))
                            
                                if is_overlapping(original_bounding_box, pot_street_bounding_box):
                                    if (debug > 2):
                                        print('Matched: ' + intersection_name + ' = ' + str(pot_street_bounding_box) + ' vs ' + str(original_bounding_box))
                                    intersection_dict[intersection_name] = [float(node_lat[node_id]), float(node_lon[node_id])]
                                    if (debug > 2):
                                        print('Trace: ' + str(node_id) + ' ' + intersection_name + ' => ' + str(intersection_dict[intersection_name]))
                                loop_counter = loop_counter + 1
    
    if (debug > 0):
        print(str(objectid) + ' ' + street + ' Matching Ways: ' + str(len(way_ids)) + ' loops: ' + str(loop_counter))
    
    # Transform dictionary into list of key/value pairs
    intersection_list = []
    
    for intersection_name, intersection_details in intersection_dict.items():
        intersection_entry = [intersection_name, intersection_details[0], intersection_details[1]]
        intersection_list.append(intersection_entry)
        
    return intersection_list

Load original PBN dataset

In [ ]:
log_starting('Load original PBN dataset')

pbn_path = os.path.join(os.path.abspath(os.pardir), 'data_sources', pbn_filename)

pbn = gpd.read_file(pbn_path)

log_finished('Load original PBN dataset')

Demonstration:  Find intersections for one street

In [ ]:
find_intersections('ABBEY WALK', 'VERMONT', 'VERMONT', 'VERMONT', 0, debug=0)

Get Google Maps API connection

$6.70 AUD per 1000 requests
Refs:

https://medium.com/future-vision/google-maps-in-python-part-2-393f96196eaf
https://developers.google.com/maps/documentation/geocoding/get-api-key
https://github.com/pbugnion/gmaps/issues/79

In [ ]:
# Load GSV API key
gsv_api_key_path = os.path.join(os.path.abspath(os.pardir), gsv_api_key_filename)

print('Loading Google Street View API key from [{0:s}]'.format(gsv_api_key_path))

with open(gsv_api_key_path) as f:
    api_key = f.readline()
    f.close

# Configure a connection to the Google Maps API
gmaps.configure(api_key=api_key)

# Define a function to return the middle of a bounding box defined by two coordinates
def mid_coords(lat1, lon1, lat2, lon2):
    return ((lat1 + lat2)/2, (lon1 + lon2)/2)

Display a map of the above area.  Hard-coded coordinates are from the find_intersections example, above

If the map does not display, install with Anaconda 3:

`$ conda install -c conda-forge gmaps`

Or with pip:

`$ jupyter nbextension enable --py --sys-prefix widgetsnbextension`

`$ pip install gmaps`

`$ jupyter nbextension enable --py --sys-prefix gmaps`

In [ ]:
gmaps.figure(center=mid_coords(-37.8388277, 145.2113893, -37.8423446, 145.2132833), zoom_level=14)

## Find distinct [local_street] x [town] combinations in the data

In [ ]:
# Filter to only include 'Existing' routes
pbn_likely1 = pbn_exploded[pbn_exploded['status']=='Existing']

# Filter to only include 'On Road' routes
pbn_likely2 = pbn_likely1[pbn_likely1['type']=='On Road']

# Filter to exclude 'n/a' streets
pbn_likely = pbn_likely2[pbn_likely2['local_street'] != 'n/a']

# Get distinct combinations
pbn_distinct1 = pbn_likely.groupby(['objectid', 'local_street', 'town', 'suburb', 'city']).size().reset_index().rename(columns={0:'count'})

# Get first row per objectid
pbn_distinct = pbn_distinct1.groupby('objectid').first().reset_index()

In [ ]:
# Show a sample
pbn_distinct.tail(3)

Find all intersections for each of the roads.

This is sometimes SLOW for a street.  E.g. there are 400x "ALBERT STREET" in Victoria,
therefore we have to check each occurrence of "ALBERT STREET" in the dataset to EVERY
street that intersects ANY "ALBERT STREET" in Victoria.  And we need to call the Nominatim
web service to get the bounding box for each street involved.

In [ ]:
# Test run for just the first 6 entries, with debug messages
log_starting('Find all intersections')

tqdm.pandas()

pbn_sample = pbn_distinct[:6].copy()
pbn_sample.loc[:, 'intersections'] = pbn_sample.progress_apply(lambda x: find_intersections(x['local_street'], x['town'], x['suburb'], x['city'], objectid=x['objectid'], debug=1), axis=1)


log_finished('Find all intersections')

pbn_sample.head()
print(pbn_sample['intersections'][0])

In [ ]:
pbn_sample.head()

In [ ]:
# Full run
log_starting('Find all intersections')

tqdm.pandas()

pbn_full = pbn_distinct.copy()
pbn_full['intersections'] = pbn_full.progress_apply(lambda x: find_intersections(x['local_street'], x['town'], x['suburb'], x['city'], objectid=x['objectid']), axis=1)

log_finished('Find all intersections')

In [ ]:
pbn_full.head(5)

We now have intersection data, but we don't have a heading to use at each intersection.  This means that when we ask Google Street View for images we can't control which way the camera is pointing.  We want it to be pointing forward in the first frame, then left, right, and to the rear.

Calling the Google Street View API without a heading will give us the front view by default, but there is no way to tell it to give us e.g. the rear view relative to the car.  We must give headings relative to points on the compass.

Therefore, before we save the output CSV, we want to work out a heading at each intersection.

Define a function to work out the heading at each intersection.  We can use a Geodesic function to calculate the heading from one point to the next.  We therefore assume that the heading at an intersection is the average of:

* The heading from the previous node to the intersection, and
* The heading from the intersection to the next node

If there is no previous node, or there is no next node, we just take the heading from what we do have.

In [ ]:
def find_bearing_and_distance(objectid, intersection):
    if type(intersection) != list:
        return None
    
    lat = intersection[1]
    lon = intersection[2]
    
    #print('int type: ' + str(type(intersection)) + ' value: ' + str(intersection))
    #print('lat type: ' + str(type(lat)) + ' value: ' + str(lat))
    #print('lon type: ' + str(type(lon)) + ' value: ' + str(lat))
    
    row = pbn[pbn['objectid']==objectid]
    
    gs = row['geometry']
    
    # Flatten into list of coordinates
    coords_list = []
    
    for g in gs:
        if type(g) is LineString:
            for xy in g.coords:
                coords_list.append((xy[1], xy[0]))
        elif type(g) is MultiLineString:
            for ls in g:
                for xy in ls.coords:
                    coords_list.append((xy[1], xy[0]))
    
    # Find the index of the closest point, and the distance in metres
    coords_this  = (lat, lon)
    
    min_distance = 20000000
    min_i        = -1
    
    for i in range(len(coords_list)):
        distance = geopy.distance.distance(coords_this, coords_list[i]).m
        
        if distance < min_distance:
            min_distance = distance
            min_i        = i

    # Get bearing to previous and next point (if applicable)
    if i > 0:
        coords_prev = coords_list[i-1]
        bearing_prev = Geodesic.WGS84.Inverse(coords_prev[0], coords_prev[1], lat, lon)['azi1']
        if bearing_prev < 0:
            bearing_prev = bearing_prev + 360
    else:
        bearing_prev = None
        
    if i < len(coords_list)-1:
        coords_next = coords_list[i+1]
        bearing_next = Geodesic.WGS84.Inverse(lat, lon, coords_next[0], coords_next[1])['azi1']
        if bearing_next < 0:
            bearing_next = bearing_next + 360
    else:
        bearing_next = None
        
    if   bearing_prev is None and bearing_next is not None:
        bearing = round(bearing_next)
    elif bearing_next is None and bearing_prev is not None:
        bearing = round(bearing_prev)
    else:
        bearing = round((bearing_prev + bearing_next) / 2)
    
    # Return a list:
    # 0: Bearing
    # 1: Distance to closest point
    # 2: Coordinates (lat, lon) of closest point
    
    return [bearing, min_distance, coords_list[i][0], coords_list[i][1]]

# Example usage
find_bearing_and_distance(1, [0, -38.1990811, 145.1044225])

In [ ]:
# Perform a test run of attempting to get a heading/bearing for each coordinate, with the sample data from earlier

In [ ]:
pbn_sample2 = pbn_sample.explode('intersections')

In [ ]:
pbn_sample2.head()

In [ ]:
log_starting('Find bearings')

tqdm.pandas()

pbn_sample3 = pbn_sample2.copy()
pbn_sample3.loc[:, 'bearings'] = pbn_sample3.progress_apply(lambda x: \
    find_bearing_and_distance(x['objectid'], x['intersections']), axis=1)

log_finished('Find bearings')

pbn_sample3.head()
#print(pbn_sample3['bearings'][0])


In [ ]:
pd.set_option('display.max_colwidth', 100)
print(pbn_sample3['bearings'])

In [ ]:
find_intersections('HYDE STREET', 'Seddon', 'Footscray', 'Melbourne', 616, debug=0)

Find bearings at each intersection

In [ ]:
pbn_full2 = pbn_full.explode('intersections')

In [ ]:
log_starting('Find bearings')

tqdm.pandas()

pbn_full3 = pbn_full2.copy()
pbn_full3.loc[:, 'bearings'] = pbn_full3.progress_apply(lambda x: \
    find_bearing_and_distance(x['objectid'], x['intersections']), axis=1)

log_finished('Find bearings')

pbn_full3.head()

Show the results just for the town of 'Mount Eliza'

In [ ]:
pbn_full3[pbn_full3['town'] == 'Mount Eliza']

Extracts parts of the 'geocode_list' column to separate columns

In [ ]:
# Extract or derive geo fields from geocode_list

def get_list_field(intersection, index):
    if not (type(intersection)==list):
        return None
    return intersection[index]
            
log_starting('Extract intersection and bearing fields')

tqdm.pandas()

pbn_full4 = pbn_full3.copy()
pbn_full4.loc[:, 'intersection_street'] = pbn_full4.progress_apply(lambda x: get_list_field(x['intersections'], 0), axis=1)
pbn_full4.loc[:, 'intersection_lat']    = pbn_full4.progress_apply(lambda x: get_list_field(x['intersections'], 1), axis=1)
pbn_full4.loc[:, 'intersection_lon']    = pbn_full4.progress_apply(lambda x: get_list_field(x['intersections'], 2), axis=1)
pbn_full4.loc[:, 'bearing']             = pbn_full4.progress_apply(lambda x: get_list_field(x['bearings'], 0), axis=1)
pbn_full4.loc[:, 'bearing_lat']         = pbn_full4.progress_apply(lambda x: get_list_field(x['bearings'], 1), axis=1)
pbn_full4.loc[:, 'bearing_lon']         = pbn_full4.progress_apply(lambda x: get_list_field(x['bearings'], 2), axis=1)
pbn_full4.drop(labels=['intersections', 'bearings'], axis=1, inplace=True)

log_finished('Extract intersection and bearing fields')
pbn_full4.head()

Save the final output

In [ ]:
log_starting('Save intersections from PBN routes')

output_intersections_path = os.path.join(os.path.abspath(os.pardir), 'data_sources', output_intersections_file)

pbn_full4.to_csv(output_intersections_path)

log_finished('Save intersections from PBN routes')